In [4]:
import numpy as np

class HierarchicalClustering:
    def __init__(self, n_clusters=2):
        self.n_clusters = n_clusters
        self.labels_ = None

    def fit_predict(self, X):
        # Initialize clusters as each data point being its own cluster
        clusters = [[i] for i in range(X.shape[0])]

        # Compute the initial distance matrix
        distances = self._compute_distance_matrix(X)

        while len(clusters) > self.n_clusters:
            # Find the two closest clusters
            i, j = self._find_closest_clusters(distances)

            # Merge the two closest clusters
            clusters[i].extend(clusters[j])
            clusters.pop(j)

            # Update the distance matrix
            distances = self._update_distance_matrix(distances, clusters, X)

        # Assign labels based on the final clusters
        self.labels_ = np.zeros(X.shape[0], dtype=int)
        for cluster_idx, cluster in enumerate(clusters):
            for sample_idx in cluster:
                self.labels_[sample_idx] = cluster_idx

        return self.labels_

    def _compute_distance_matrix(self, X):
        """Computes the initial distance matrix using Euclidean distance."""
        n_samples = X.shape[0]
        distances = np.full((n_samples, n_samples), np.inf)  # Initialize with infinity

        for i in range(n_samples):
            for j in range(i + 1, n_samples):
                distances[i, j] = np.linalg.norm(X[i] - X[j])
                distances[j, i] = distances[i, j]  # Symmetric distance

        return distances

    def _find_closest_clusters(self, distances):
        """Finds the two closest clusters based on the distance matrix."""
        min_dist = np.inf
        closest_pair = (0, 0)

        for i in range(len(distances)):
            for j in range(i + 1, len(distances)):
                if distances[i, j] < min_dist:
                    min_dist = distances[i, j]
                    closest_pair = (i, j)

        return closest_pair

    def _update_distance_matrix(self, distances, clusters, X):
        """Updates the distance matrix after merging two clusters."""
        n_clusters = len(clusters)
        new_distances = np.full((n_clusters, n_clusters), np.inf)

        for i in range(n_clusters):
            for j in range(i + 1, n_clusters):
                # Compute distance between cluster i and cluster j
                dist = self._calculate_cluster_distance(clusters[i], clusters[j], X)
                new_distances[i, j] = dist
                new_distances[j, i] = dist  # Symmetric distance

        return new_distances

    def _calculate_cluster_distance(self, cluster1, cluster2, X):
        """Calculates the distance between two clusters (using single linkage)."""
        min_distance = np.inf

        for idx1 in cluster1:
            for idx2 in cluster2:
                distance = np.linalg.norm(X[idx1] - X[idx2])
                if distance < min_distance:
                    min_distance = distance

        return min_distance

# Example usage
X = np.array([
    [2, 3], [1, 1], [2, 2], [4, 4], [6, 6], [7, 8], [1, 3], [3, 3],
    [5, 5], [8, 8], [3, 7], [9, 3], [6, 2], [4, 5], [5, 7], [2, 6],
    [3, 6], [7, 9], [8, 7], [5, 6], [6, 4], [7, 5], [2, 5], [4, 6],
    [1, 2], [8, 3], [3, 8], [2, 4], [6, 5], [9, 6], [4, 2], [5, 4]
])

# Initialize the model
hierarchical_clustering = HierarchicalClustering(n_clusters=3)
clusters = hierarchical_clustering.fit_predict(X)

print("Cluster assignments:", clusters)


Cluster assignments: [0 0 0 0 0 1 0 0 0 1 0 2 0 0 0 0 0 1 1 0 0 0 0 0 0 2 0 0 0 1 0 0]
